Exercise 2 
In this chapter, I showed how we can express the DFT and inverse DFT as matrix multiplications. These operations take time proportional to N2, where N is the length of the wave array. That is fast enough for many applications, but there is a faster algorithm, the Fast Fourier Transform (FFT), which takes time proportional to N logN.
The key to the FFT is the Danielson-Lanczos lemma:
DFT(y)[n] = DFT(e)[n] + exp(−2 π i n / N) DFT(o)[n]
Where DFT(y)[n] is the nth element of the DFT of y; e is a wave array containing the even elements of y, and o contains the odd elements of y.
This lemma suggests a recursive algorithm for the DFT:
1. Given a wave array, y, split it into its even elements, e, and its odd elements, o.
2. Compute the DFT of e and o by making recursive calls.
3. Compute DFT(y) for each value of n using the Danielson-Lanczos lemma.
For the base case of this recursion, you could wait until the length of y is 1. In that case, DFT(y) = y. Or if the length of y is sufficiently small, you could compute its DFT by matrix multiplication, possibly using a precomputed matrix.
Hint: I suggest you implement this algorithm incrementally by starting with a version that is not truly recursive. In Step 2, instead of making a recursive call, use dft, as defined in Section 7.7, or np.fft.fft. Get Step 3 working, and confirm that the results are consistent with the other implementations. Then add a base case and confirm that it works. Finally, replace Step 2 with recursive calls.
One more hint: Remember that the DFT is periodic; you might find np.tile useful.
You can read more about the FFT at https://en.wikipedia.org/wiki/Fast_Fourier_transform.

在本章中，我展示瞭如何將DFT和逆DFT表示為矩陣乘法。這些操作花費的時間與N2成正比，其中N是波陣的長度。對於許多應用而言，這足夠快，但是有一種更快的算法，即快速傅立葉變換（FFT），它花費的時間與N logN成正比。
FFT的關鍵是Danielson-Lanczos引理：
DFT(y)[n] = DFT(e)[n] + exp(−2 π i n / N) DFT(o)[n]
其中DFT（y）[n]是y的DFT的第n個元素； e是包含y偶數元素的波陣，而o包含y奇數元素的波陣。
這個引理提出了DFT的遞歸算法：
1.給定一個波數組y，將其分成偶數元素e和奇數元素o。
2.通過進行遞歸調用來計算e和o的DFT。
3.使用Danielson-Lanczos引理為n的每個值計算DFT（y）。
對於此遞歸的基本情況，您可以等待y的長度為1。在這種情況下，DFT（y）= y。或者，如果y的長度足夠小，則可以通過矩陣乘法（可能使用預先計算的矩陣）來計算其DFT。
提示：我建議您從不是真正遞歸的版本開始逐步實現此算法。在第2步中，而不是進行遞歸調用，而是使用7.7節中定義的dft或np.fft.fft。使第3步正常工作，並確認結果與其他實現一致。然後添加一個基本案例並確認其有效。最後，將第2步替換為遞歸調用。
還有一個提示：請記住DFT是周期性的。您可能會發現np.tile有用。

In [1]:
import numpy as np
PI2 = 2 * np.pi

我將從一個小的實信號開始，併計算其FFT

In [2]:
ys = [-0.5, 0.1, 0.7, -0.1]
hs = np.fft.fft(ys)
print(hs)

[ 0.2+0.j  -1.2-0.2j  0.2+0.j  -1.2+0.2j]


In [3]:
def dft(ys):
    N = len(ys)
    ts = np.arange(N) / N
    freqs = np.arange(N)
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    amps = M.conj().transpose().dot(ys)
    return amps

In [4]:
hs2 = dft(ys)
np.sum(np.abs(hs - hs2))

5.864775846765962e-16

In [5]:
def fft_norec(ys):
    N = len(ys)
    He = np.fft.fft(ys[::2])
    Ho = np.fft.fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

In [6]:
hs3 = fft_norec(ys)
np.sum(np.abs(hs - hs3))

0.0

最後，我們可以將np.fft.fft替換為遞歸調用，並添加一個基本案例：

In [7]:
def fft(ys):
    N = len(ys)
    if N == 1:
        return ys
    
    He = fft(ys[::2])
    Ho = fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

In [8]:
hs4 = fft(ys)
np.sum(np.abs(hs - hs4))

1.6653345369377348e-16

FFT的這種實現所花費的時間與𝑛log𝑛成比例。它還佔用與𝑛log𝑛成比例的空間，並且浪費一些時間來製作和復制數組。可以改進以“就地”運行；在這種情況下，它不需要額外的空間，並且花費的時間更少。